In [1]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from math import sqrt
from sklearn.model_selection import KFold
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

plt.rcParams['figure.figsize'] = [16, 10]

In [2]:
df = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')


In [4]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [7]:
X = df[[ 'total_households', 'total_individuals', 'dw_00',
       'dw_01', 'dw_02', 'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08',
       'dw_09', 'dw_10', 'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01',
       'psa_02', 'psa_03', 'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01',
       'lln_00', 'lln_01', 'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04',
       'lan_05', 'lan_06', 'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11',
       'lan_12', 'lan_13', 'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03',
       'pg_04', 'lgt_00', 'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05',
       'pw_06', 'pw_07', 'pw_08',  'lat', 'lon', 'NL']]

y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
sel_ = SelectFromModel(Lasso(alpha=0.05, random_state=0))
sel_.fit(X_train, y_train)
sel_.get_support()

SelectFromModel(estimator=Lasso(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)

In [11]:
selected_feates = X_train.columns[sel_.get_support()]
print("total features: {}". format(X_train.shape[1]))
print("selected fetaures: {}". format(len(selected_feates)))
print("features with coefficients shrank to zero:{}". 
      format(np.sum(sel_.estimator_==0)))

selected_feates

total features: 60
selected fetaures: 19
features with coefficients shrank to zero:0


In [13]:
X = df[['lln_00','total_individuals', 'lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', 'lon',
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']]

y= df["target"]

Xtest= df_test[['lln_00', 'total_individuals','lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', 'lon',
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']]

In [ ]:
new subsets using lasso regression

In [13]:
X = df[['total_households', 'total_individuals', 'dw_00', 'dw_01', 'dw_08',
       'psa_00', 'psa_01', 'car_00', 'lan_00', 'lan_04', 'lan_05', 'lan_06',
       'lan_11', 'lgt_00', 'pw_00', 'pw_01', 'lat', 'lon', 'NL']]
y = df["target"]
Xtest = df_test[['total_households', 'total_individuals', 'dw_00', 'dw_01', 'dw_08',
       'psa_00', 'psa_01', 'car_00', 'lan_00', 'lan_04', 'lan_05', 'lan_06',
       'lan_11', 'lgt_00', 'pw_00', 'pw_01', 'lat', 'lon', 'NL']]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

/usr/lib/python3/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [45]:
rf = RandomForestRegressor(n_estimators=333, min_samples_split=2, max_features= 3,
                           min_samples_leaf=2,max_depth=110, bootstrap= False)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=110,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=333, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [54]:
predictions= rf.predict(X_test)
print("RMSE:", sqrt(mean_squared_error(y_test, predictions)))

RMSE: 3.3482664460513396


In [19]:
df_test["new_preds"]=rf.predict(Xtest)

In [20]:
df_test[['ward',"new_preds"]].to_csv('new_predictions.csv', index=False)

In [21]:
df_test[['ward', "new_preds"]]

,ward,new_preds
0,21001001: Ward 1,18.072972
1,21001002: Ward 2,13.686865
2,21001003: Ward 3,21.207560
3,21001004: Ward 4,16.741944
4,21001005: Ward 5,21.055063
...,...,...
1008,19100107: Ward 107,8.534250
1009,19100108: Ward 108,20.247444
1010,19100109: Ward 109,17.020437
1011,19100110: Ward 110,11.916006


## XGBOOST REGRESSOR MODEL

In [44]:
model_xgb = XGBRegressor(n_estimators=600,learning_rate=0.05, min_child_weight=2,max_depth=15,
                         eta =0.6, reg_lambda = 1.2)
model_xgb.fit(X_train, y_train)
sqrt(mean_squared_error(y_test, model_xgb.predict(X_test)))

[20:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


3.2675435446614873

In [65]:
create_submission(df_test, 'xgboost_trial.csv', cols=top_cols_20, models=[model_xgb])

,ward,target
0,21001001: Ward 1,19.703636
1,21001002: Ward 2,12.059702
2,21001003: Ward 3,19.268044
3,21001004: Ward 4,17.389244
4,21001005: Ward 5,21.272993
...,...,...
1008,19100107: Ward 107,5.668094
1009,19100108: Ward 108,21.461271
1010,19100109: Ward 109,15.091393
1011,19100110: Ward 110,12.040772


### Best parameters

{'colsample_bytree': 0.8, 'max_depth': 20, 'n_estimators': 1000, 'reg_alpha': 1.3, 'reg_lambda': 1.3, 'subsample': 0.7}




## Parameter tunning for xgboost

In [41]:
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     StratifiedKFold)

In [42]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=3, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [43]:
model = XGBRegressor()
param_grid = {
    'n_estimators': [200, 300, 600],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'subsample': [0.7, 0.8, 0.9]
}

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid, cv=5)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   24.4s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done 359 tasks      | elapsed:  5.7min
[Parallel(n_jobs=3)]: Done 642 tasks      | elapsed:  9.9min
[Parallel(n_jobs=3)]: Done 1007 tasks      | elapsed: 16.4min
[Parallel(n_jobs=3)]: Done 1452 tasks      | elapsed: 24.2min
[Parallel(n_jobs=3)]: Done 1979 tasks      | elapsed: 34.7min
[Parallel(n_jobs=3)]: Done 2430 out of 2430 | elapsed: 44.1min finished
/home/aims/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:48:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
3.460746992995065
{'colsample_bytree': 0.7, 'max_depth': 15, 'n_estimators': 600, 'reg_alpha': 1.1, 'reg_lambda': 1.2, 'subsample': 0.7}


In [47]:
model = RandomForestRegressor()
param_grid = {
    'n_estimators': [100, 300, 600],
    "min_samples_split": [2, 3,6],
    'max_depth': [15,20,25],
    'min_samples_leaf': [1, 2, 3],
    'max_features': [1,2, 3],
    'bootstrap': [True, False]
}

model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
                                 param_grid, cv=5)

print(np.sqrt(-model.best_score_))
print(model.best_params_)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    7.9s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:   31.7s
[Parallel(n_jobs=3)]: Done 359 tasks      | elapsed:  1.7min
[Parallel(n_jobs=3)]: Done 642 tasks      | elapsed:  3.0min
[Parallel(n_jobs=3)]: Done 1007 tasks      | elapsed:  4.9min
[Parallel(n_jobs=3)]: Done 1452 tasks      | elapsed:  7.5min
/usr/lib/python3/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=3)]: Done 1979 tasks      | elapsed: 12.0min
[Parallel(n_jobs=3)]: Done 2430 out of 2430 | elapsed: 16.0min finished


3.487941906337393
{'bootstrap': False, 'max_depth': 25, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 600}


## Combined xgboost and random forest

In [55]:
avg_pre = (model_xgb.predict(X_test) + rf.predict(X_test))/2

sqrt(mean_squared_error(y_test, avg_pre))

3.2044422469045304

## Save in csv 

In [24]:
def create_submission(data, fname, cols, models=[]):
    score = 0
    for model in models:
        score += model.predict(data[cols])
    data['target'] = (score / len(models))
    submissions = data[['ward', 'target']]
    submissions.to_csv(fname, index=False)
    return submissions

In [20]:
top_cols_20 = ['lln_00', 'total_individuals','lan_06', 'psa_04', 'lat', 'lan_03', 'lan_07', 'psa_01',
       'lan_09', 'dw_01', 'lan_05', 'NL', 'lln_01', 'pg_00', 'lan_11', 'lon',
       'lan_10', 'car_00', 'car_01', 'pw_00', 'psa_00']

In [27]:
create_submission(df_test, 'randomf_xgboost_reduced.csv', cols=top_cols_20, models=[rf, model_xgb])

,ward,target
0,21001001: Ward 1,18.653955
1,21001002: Ward 2,13.045864
2,21001003: Ward 3,20.486401
3,21001004: Ward 4,15.844018
4,21001005: Ward 5,21.146926
...,...,...
1008,19100107: Ward 107,8.080331
1009,19100108: Ward 108,19.902534
1010,19100109: Ward 109,16.400654
1011,19100110: Ward 110,12.320537
